# Mask analysis full pipeline for level2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys, os
#from cellpose import io
from skimage import io
from skimage.measure import label, regionprops
import itertools

other_dir_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..','..'))
sys.path.append(other_dir_path)

import IMCprocess.spatial_analysis.mask_utils as mu
import IMCprocess.utils.utils as ut

## Merge label for level2 in level1 object

In [32]:
data_path = '/Users/pattama/Desktop/UiB/cancer/example_share_code/share_with_vladan/mask_analysis/example_data/'
data_lev1 = ut.read_pickle_obj(data_path+'00hanna_adata_totalNB_obj_Level1.pkl')
Ep_lev2 = ut.read_pickle_obj(data_path+'02hanna_adata_totalNB_obj_EpLevel2.pkl')
Im_lev2 = ut.read_pickle_obj(data_path+'02hanna_adata_totalNB_obj_ImmuneLevel2.pkl')
Fi_lev2 = ut.read_pickle_obj(data_path+'02hanna_adata_totalNB_obj_Level2.pkl')

In [33]:
# get anndata for level1, epithelial level2, immune level2, fibroblast level2
level1_adata = data_lev1.adata_dict['level1']
Ep_adata = Ep_lev2.adata_dict['Ep_level2']
Im_adata = Im_lev2.adata_dict['Immune_level2']
Fi_adata = Fi_lev2.adata_dict['Fibro_level2']

In [ ]:
def add_level2_to_level1_obs(level1_adata, level2_adata_list):
    '''
    Add level2 annotation to level1 adata object
    '''
    level1_adata.obs['level2'] = level1_adata['level1_annotated'].astype(str)
    for lev2_adata in level2_adata_list:
        lev2_col = lev2_adata.obs.columns[lev2_adata.obs.columns.str.contains('level2')][0]
        lev2_adata.obs['level2'] = lev2_adata.obs[
            'level1_annotated'].astype('str') + '_' + lev2_adata.obs[lev2_col].astype('str')
        level1_adata.obs.loc[lev2_adata.obs.level2.index, 'level2'] = lev2_adata.obs.level2
    return level1_adata

In [47]:
#Ep_adata.obs.columns.to_list().contains('level2')
Ep_adata.obs.columns[Ep_adata.obs.columns.str.contains('level2')][0]

'Ep_level2_annotated'

In [4]:
data_lev1.adata_dict

{'level1': AnnData object with n_obs × n_vars = 132951 × 51
     obs: 'img_id', 'PhenoGraph_clusters', 'level1_annotated'
     uns: 'pca', 'PhenoGraph_Q', 'PhenoGraph_k', 'neighbors', 'umap', 'PCs', 'dendrogram_PhenoGraph_clusters', 'PhenoGraph_clusters_colors', 'level1_annotated_colors'
     obsm: 'spatial', 'X_pca', 'X_umap'
     obsp: 'distances', 'connectivities'}

In [5]:
Ep_lev2.adata_dict

{'level1': AnnData object with n_obs × n_vars = 132951 × 51
     obs: 'img_id', 'PhenoGraph_clusters', 'level1_annotated'
     uns: 'pca', 'PhenoGraph_Q', 'PhenoGraph_k', 'neighbors', 'umap', 'PCs', 'dendrogram_PhenoGraph_clusters', 'PhenoGraph_clusters_colors', 'level1_annotated_colors'
     obsm: 'spatial', 'X_pca', 'X_umap'
     obsp: 'distances', 'connectivities',
 'Ep_level2': AnnData object with n_obs × n_vars = 64874 × 51
     obs: 'img_id', 'PhenoGraph_clusters', 'level1_annotated', 'Ep_level2_annotated'
     uns: 'pca', 'PhenoGraph_Q', 'PhenoGraph_k', 'neighbors', 'umap', 'PCs', 'dendrogram_PhenoGraph_clusters', 'PhenoGraph_clusters_colors', 'level1_annotated_colors', 'Ep_level2_annotated_colors'
     obsm: 'spatial', 'X_pca', 'X_umap'
     obsp: 'distances', 'connectivities'}

In [6]:
Im_lev2.adata_dict

{'level1': AnnData object with n_obs × n_vars = 132951 × 51
     obs: 'img_id', 'PhenoGraph_clusters', 'level1_annotated'
     uns: 'pca', 'PhenoGraph_Q', 'PhenoGraph_k', 'neighbors', 'umap', 'PCs', 'dendrogram_PhenoGraph_clusters', 'PhenoGraph_clusters_colors', 'level1_annotated_colors'
     obsm: 'spatial', 'X_pca', 'X_umap'
     obsp: 'distances', 'connectivities',
 'Immune_level2': AnnData object with n_obs × n_vars = 29445 × 51
     obs: 'img_id', 'PhenoGraph_clusters', 'level1_annotated', 'Immune_level2_annotated'
     uns: 'pca', 'PhenoGraph_Q', 'PhenoGraph_k', 'neighbors', 'umap', 'PCs', 'dendrogram_PhenoGraph_clusters', 'PhenoGraph_clusters_colors', 'level1_annotated_colors', 'Immune_level2_annotated_colors'
     obsm: 'spatial', 'X_pca', 'X_umap'
     obsp: 'distances', 'connectivities'}

In [7]:
Fi_lev2.adata_dict

{'level1': AnnData object with n_obs × n_vars = 132951 × 51
     obs: 'img_id', 'PhenoGraph_clusters', 'level1_annotated'
     uns: 'pca', 'PhenoGraph_Q', 'PhenoGraph_k', 'neighbors', 'umap', 'PCs', 'dendrogram_PhenoGraph_clusters', 'PhenoGraph_clusters_colors', 'level1_annotated_colors'
     obsm: 'spatial', 'X_pca', 'X_umap'
     obsp: 'distances', 'connectivities',
 'Fibro_level2': AnnData object with n_obs × n_vars = 17129 × 51
     obs: 'img_id', 'PhenoGraph_clusters', 'level1_annotated', 'Fibro_level2_annotated'
     uns: 'pca', 'PhenoGraph_Q', 'PhenoGraph_k', 'neighbors', 'umap', 'PCs', 'dendrogram_PhenoGraph_clusters', 'PhenoGraph_clusters_colors', 'level1_annotated_colors', 'Fibro_level2_annotated_colors'
     obsm: 'spatial', 'X_pca', 'X_umap'
     obsp: 'distances', 'connectivities'}

In [12]:
lev1 = data_lev1.adata_dict['level1'].obs

In [15]:
lev1

,img_id,PhenoGraph_clusters,level1_annotated
Cell_id,,,
1_0,R1_001,45,Ep
2_0,R1_001,12,Immune
3_0,R1_001,34,Ves
4_0,R1_001,0,Fibro
5_0,R1_001,45,Ep
...,...,...,...
130701_29,R10_003,13,Ep
130702_29,R10_003,28,Ep
130703_29,R10_003,1,Ep


In [13]:
ep = Ep_lev2.adata_dict['Ep_level2'].obs

In [26]:
lev1['level2'] = lev1['level1_annotated'].astype(str)

In [18]:
lev1.loc['1_0', 'level2']='

'Ep'

In [22]:
ep['level2'] = ep['level1_annotated'].astype('str') + '_' + ep['Ep_level2_annotated'].astype('str')
ep.head()

,img_id,PhenoGraph_clusters,level1_annotated,Ep_level2_annotated,level2
Cell_id,,,,,
1_0,R1_001,19,Ep,Ca_nonstem,Ep_Ca_nonstem
5_0,R1_001,19,Ep,Ca_nonstem,Ep_Ca_nonstem
6_0,R1_001,11,Ep,Ca_nonstem,Ep_Ca_nonstem
7_0,R1_001,35,Ep,Ca_nonstem,Ep_Ca_nonstem
8_0,R1_001,30,Ep,Ca_nonstem,Ep_Ca_nonstem


In [23]:
ep.level2

Cell_id
1_0          Ep_Ca_nonstem
5_0          Ep_Ca_nonstem
6_0          Ep_Ca_nonstem
7_0          Ep_Ca_nonstem
8_0          Ep_Ca_nonstem
                 ...      
130698_29    Ep_Ca_nonstem
130701_29    Ep_Ca_nonstem
130702_29    Ep_Ca_nonstem
130703_29    Ep_Ca_nonstem
130704_29    Ep_Ca_nonstem
Name: level2, Length: 64874, dtype: object

In [27]:
lev1.loc[ep.level2.index, 'level2'] = ep.level2

In [28]:
lev1.head()

,img_id,PhenoGraph_clusters,level1_annotated,level2
Cell_id,,,,
1_0,R1_001,45,Ep,Ep_Ca_nonstem
2_0,R1_001,12,Immune,Immune
3_0,R1_001,34,Ves,Ves
4_0,R1_001,0,Fibro,Fibro
5_0,R1_001,45,Ep,Ep_Ca_nonstem
